In [50]:
import pandas as pd

In [3]:
original = pd.read_excel('./original.xlsx')

In [15]:
original.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [28]:
original_ = original.drop(columns=['InvoiceNo', 'Quantity', 'UnitPrice'])

In [30]:
original_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   StockCode    541909 non-null  object        
 1   Description  540455 non-null  object        
 2   InvoiceDate  541909 non-null  datetime64[ns]
 3   CustomerID   406829 non-null  float64       
 4   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 20.7+ MB


In [39]:
original_[original_['CustomerID'] == 17850.0].head(5)

,StockCode,Description,InvoiceDate,CustomerID,Country
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,17850.0,United Kingdom
1,71053,WHITE METAL LANTERN,2010-12-01 08:26:00,17850.0,United Kingdom
2,84406B,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,17850.0,United Kingdom
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,17850.0,United Kingdom
4,84029E,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,17850.0,United Kingdom


# need to get what every Customer bought => unique by CustomerID
## but a customer can make many transaction in defferent times 
# unique by time such that all transactions made in the range of 15 minutes will be considered as one transaction

In [95]:
# the time interval for transaction grouping in minutes
time_interval = 15

transactions = []

for _, customer_ in original_.groupby('CustomerID'):
    transaction = []
    reference_time = None

    for index, row in customer_.iterrows():
        time = str(row['InvoiceDate']).split(' ')[1]
        hour = int(time[:2])
        minute = int(time[3:5])

        if reference_time is None:
            reference_time = {"h": hour, "m": minute}

        if hour == reference_time['h'] and abs(minute - reference_time['m']) <= time_interval:
            transaction.append(row['StockCode'])
        else:
            transactions.append(transaction)
            transaction = [row['StockCode']]
            reference_time = {"h": hour, "m": minute}

    transactions.append(transaction)
    

In [97]:
apriory_df = pd.DataFrame(transactions)

In [100]:
apriory_df.to_excel('./transactions.xlsx', index=False)